In [1]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from PIL import Image
import os
from tqdm import tqdm

# torch.backends.cuda.matmul.allow_tf32 = False
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from ultralytics import YOLO
import torch.nn.functional as F
from tqdm import trange
import albumentations as A
threshold = [0, 0, 0]
import json

In [2]:
import sys
sys.path.append('../utils/')
# sys.path.append('../utils/model/')
from generateMaskArea import generate_mask_area
from bboxAnalyse import yolov8_detection, get_score, record, get_GT_label,convert_scores,cal_correct_count,show_class_item

from getFrameNumber import get_total_frame_number

In [3]:
class CFG:
    ######### importtant #########
    video_name = 'GX020091_resize.mp4'
    binary_model_addr = 'D:/model56.pth'
    video_root =  'D:/data/cow_teat_segmentation/2022_02_08_Aim2-selected'
    save_root = 'D:/2022_02_08_Aim2-selected_mask_area'
    yolo_model_path = '../model/yolov8m.pt'
    ######### importtant #########
    
    if os.path.exists(save_root) == False:
        os.mkdir(save_root)
    
    
    device = 'cuda'
    
    
    video_list = os.listdir(video_root)
    video_name_list = [i.split('_')[0] for i in video_list]

    video_path = video_root + '/' + video_name
    transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2(),
    ])
cfg = CFG()

# binary segmentation model

In [4]:
from model.unet import UNet

In [5]:
model = UNet(2)
model.load_state_dict(torch.load(cfg.binary_model_addr))
model.to(cfg.device)

UNet(
  (inc): DoubleConv(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

# generate mask area image

In [6]:

generate_mask_area(model,device=cfg.device, save_root=cfg.save_root, video_path = cfg.video_path,Atransform=cfg.transform)

Video GX020091_resize is already processed


In [7]:
image_pre_name = cfg.video_name.split('.')[0] + '_'
root_dir = cfg.save_root + '/' + cfg.video_name.split('.')[0] +'/'

In [8]:
class SegmentationDataset():
  def __init__(self, root_dir, image_pre_name,  transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.image_list = os.listdir(root_dir)
    self.len = len(self.image_list)
    self.image_pre_name = image_pre_name # 'GX010089_resize_' or 'GX030089_resize_'
  def __len__(self):
    return self.len
    # return len(self.image_list)

  def __getitem__(self, idx):
    image_name = self.image_pre_name + str(idx)+'.png'
    img_path= self.root_dir+ image_name
    image = cv2.imread(img_path)
    return image,img_path

In [9]:
data = SegmentationDataset(root_dir = root_dir, image_pre_name=image_pre_name)

In [10]:
yolo_model  = YOLO(cfg.yolo_model_path)

In [11]:
all_result = dict()

In [12]:
# for video_name in GT_labels:
#     if video_name in all_result:
#         print(video_name + " already processed")
#         continue
#     mask_aadr = cfg.save_root + '/' + video_name + "_resize/"
#     image_pre_name = video_name + '_resize_'
#     mask_dataset = SegmentationDataset(root_dir = mask_aadr, image_pre_name=image_pre_name)
scores = np.array([[[0]*4]*4]*61)

video_path = cfg.video_root+"/"+cfg.video_name

count = 0 
video = cv2.VideoCapture(video_path)
if (video.isOpened()== False):
    print("Error opening video file")
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

with tqdm(total=total_frames,desc="processing video " + cfg.video_name ) as pbar:
    while(video.isOpened()):
    # Capture frame-by-frame
        pbar.update(1)
        ret, original_image = video.read()
        if ret == True:
            image, img_path = data[count]
            count += 1
            result = np.nan*np.ones(65)
            mask = np.all(image > threshold, axis=2)
            yolov8_boxex,yolov8_class_id, image,results = yolov8_detection(yolo_model, image)
            
            copy_oirginal_image = original_image.copy()
            if len(yolov8_boxex) > 0:
                score_for_1img = get_score(yolov8_class_id,yolov8_boxex)
                record(scores,score_for_1img)
            else:
                continue
        else:
            break
    video.release()
all_result[cfg.video_name.split('_')[0]] = convert_scores(scores)
# save as json
with open('scores.json', 'w') as fp:
    json.dump(all_result, fp)

processing video GX020091_resize.mp4: 15961it [04:29, 59.18it/s]                           


In [27]:
result

{'GX010089': {'1': [nan, nan, nan, nan],
  '2': [nan, nan, nan, nan],
  '3': [nan, nan, nan, nan],
  '4': [nan, nan, nan, nan],
  '5': [nan, nan, nan, nan],
  '6': [nan, nan, nan, nan],
  '7': [nan, nan, nan, nan],
  '8': [nan, nan, nan, nan],
  '9': [nan, nan, nan, nan],
  '10': [nan, nan, nan, nan],
  '11': [nan, nan, nan, nan],
  '12': [nan, nan, nan, nan],
  '13': [nan, nan, nan, nan],
  '14': [nan, nan, nan, nan],
  '15': [nan, nan, nan, nan],
  '16': [nan, nan, nan, nan],
  '17': [nan, nan, nan, nan],
  '18': [nan, nan, nan, nan],
  '19': [2.0, 4.0, 2.0, 2.0],
  '20': [nan, nan, nan, nan],
  '21': [2.0, 2.0, 2.0, 2.0],
  '22': [2.0, 2.0, 2.0, 2.0],
  '23': [nan, nan, nan, nan],
  '24': [1.0, 2.0, 1.0, 2.0],
  '25': [2.0, 2.0, 2.0, 2.0],
  '26': [2.0, 2.0, 2.0, 2.0],
  '27': [nan, nan, nan, nan],
  '28': [2.0, 2.0, 2.0, 3.0],
  '29': [2.0, 4.0, 2.0, 2.0],
  '30': [2.0, 2.0, 2.0, 2.0],
  '31': [2.0, 3.0, 2.0, 3.0],
  '32': [2.0, 2.0, 2.0, 2.0],
  '33': [3.0, 2.0, 2.0, 2.0],
  '34':

In [28]:
'GX010089' in result

True

In [22]:
mask_dataset[0]

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 'D:/data/cow_teat_segmentation/2022_02_08_Aim2-selected_mask_area/GX010089_resize/GX010089_resize_0.png')

In [9]:
data = SegmentationDataset(root_dir = root_dir, image_pre_name=image_pre_name)

In [10]:
yolo_model  = YOLO(cfg.yolo_model_path)

# calculate scores

In [11]:
scores = np.array([[[0]*4]*4]*61)

count = 0 
video = cv2.VideoCapture(cfg.video_path)
if (video.isOpened()== False):
    print("Error opening video file")
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

with tqdm(total=total_frames) as pbar:
    while(video.isOpened()):
    # Capture frame-by-frame
        ret, original_image = video.read()
        if ret == True:
            image, img_path = data[count]
            count+=1
            result = np.nan*np.ones(65)
            mask = np.all(image > threshold, axis=2)
            yolov8_boxex,yolov8_class_id, image,results = yolov8_detection(yolo_model, image)
            
            copy_oirginal_image = original_image.copy()
            if len(yolov8_boxex) > 0:
                score_for_1img = get_score(yolov8_class_id,yolov8_boxex)
                record(scores,score_for_1img)
            else:
                continue
            pbar.update(1)
        else:
            video.release()

  8%|▊         | 1233/15960 [00:43<08:44, 28.10it/s]


TypeError: '>' not supported between instances of 'NoneType' and 'list'

In [13]:
GT_dict = get_GT_label(cfg.video_name.split("_")[0])
pred_dict = convert_scores(scores)

In [18]:
total_count,correct_count,class_total,correct_class_total = cal_correct_count(GT_dict,pred_dict)
print("overall accuracy for ",cfg.video_name.split('_')[0],":",correct_count/total_count)

overall accuracy for  GX020090 : 0.44680851063829785


In [19]:
show_class_item(correct_class_total,class_total)

score 1 correct: 0.0 score 1 total: 20.0
score 2 correct: 73.0 score 2 total: 119.0
score 3 correct: 11.0 score 3 total: 33.0
score 4 correct: 0.0 score 4 total: 12.0
